In [11]:
import tensorflow as tf 
import numpy as np 
import pandas as pd
from TFData.Dataset import Dataset
from model.models import ModelCreation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import AUC,BinaryAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [4]:
IMAGE_SIZE = [1024,1024]
IMAGE_SIZE_DESIRED = [256,256]
DATASET_SIZE = 33127
BATCH_SIZE = 10
EPOCHS = 3

In [5]:
training_dataset = tf.io.gfile.glob('./Dataset/tfrecords/train*.tfrec')
test_dataset = tf.io.gfile.glob('./Dataset/tfrecords/test*.tfrec')

In [6]:
validation_dataset = training_dataset[int(0.8*len(training_dataset)):]
training_dataset = training_dataset[:int(0.8*len(training_dataset))]

In [7]:
DataGenerator = Dataset(train_files=training_dataset,
                        test_files=test_dataset,
                        validation_files=validation_dataset,
                        validation_split = 0.2,
                        image_size=IMAGE_SIZE,
                        dataset_size=DATASET_SIZE,
                        batch_size=BATCH_SIZE,
                        resize_shape=IMAGE_SIZE_DESIRED)

In [8]:
model_util = ModelCreation(architecture='efficientnet',
                      learning_rate= 0.000075,
                      input_shape=(*IMAGE_SIZE_DESIRED,3),
                      output_shape=1,
                      linear=True,
                      verbose=True,
                      loss = binary_crossentropy,
                      optimizer = Adam,
                      metric = [BinaryAccuracy(),AUC()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
efficientnet-b5 (Model)      (None, 8, 8, 2048)        28513520  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 32,711,921
Trainable params: 32,539,185
Non-trainable params: 172,736
_________________________________________________________________
Created model: efficientnet
Input Shape: (256, 256, 3)
Output Shape: 1
Mixed precission is set


In [14]:
def build_lrfn(lr_start=0.00001, lr_max=0.0001, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max 

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

lrfn = build_lrfn() 
callbacks = [
    ModelCheckpoint('my_model.h5',monitor='val_loss',verbose=1,save_best_only=True),
    LearningRateScheduler(lrfn,verbose=1)
]

In [15]:
history = model_util.model.fit(
                                DataGenerator.get_training_dataset(),
                                steps_per_epoch = DataGenerator.get_train_steps_per_epoch(),
                                epochs = EPOCHS,
                                validation_data = DataGenerator.get_validation_dataset(),
                                validation_steps = DataGenerator.get_validation_steps_per_epoch(),
                                verbose = 1,
                                callbacks=model_util.inject_callbacks(callbacks))


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/3


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[12,1056,16,16] and type half on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/efficientnet-b5/block5g_bn/FusedBatchNormV3 (defined at <ipython-input-15-a3b243b1989b>:8) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[assert_greater_equal/Assert/AssertGuard/pivot_f/_2057/_257]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[12,1056,16,16] and type half on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/efficientnet-b5/block5g_bn/FusedBatchNormV3 (defined at <ipython-input-15-a3b243b1989b>:8) ]]
Hint: If you want to see a list 

In [8]:
test_dataset = DataGenerator.get_test_dataset()
test_images = test_dataset.map(lambda image,idnum:tf.image.resize(image,[256,256]))

In [9]:
Loaded_model = load_model('my_model.h5')

In [10]:
probabilites = Loaded_model.predict(test_images)

In [12]:
NUM_TEST_IMAGES = 10982
test_ids_ds = test_dataset.map(lambda image,idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

In [13]:
sub = pd.read_csv('./Dataset/sample_submission.csv')
pred_df = pd.DataFrame({'image_name': test_ids,'target': np.concatenate(probabilites)})
del sub['target']
sub = sub.merge(pred_df,on='image_name')

In [16]:
sub.to_csv("EfficientNetB5Sub.csv",index=False)